<a href="https://colab.research.google.com/github/wisrovi/RedesNeuronales/blob/master/AprendizajePorRefuerzo-PongDeterministic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teoría

## Objetivo: 
Repasar los conceptos vistos en clase.

La puntuación de este bloque es de 4 puntos sobre la nota final.

**Define brevemente qué es el aprendizaje por refuerzo. ¿Qué diferencias hay entre aprendizaje supervisado, no supervisado y por refuerzo?**

**Define con tus palabras los conceptos de Entorno, Agente, Recompensa, Estado y Observación.**

**Dependiendo del algoritmo de aprendizaje por refuerzo que se use, ¿qué clasificaciones podemos encontrar? Coméntalas brevemente.**




**Lista tres diferencias entre los algoritmos de DQN y Policy Gradient**

# Práctica

Objetivo: Implementar una solución, usando keras-rl y basada en el algoritmo de DQN visto en clase, para que un agente aprenda una estrategia ganadora en el juego del Pong.

La puntuación de este bloque es de 6 puntos sobre la nota final.

El entorno sobre el que trabajaremos será _PongDeterministic-v0_ y el algoritmo que usaremos será _DQN_.

Para evaluar cómo lo está haciendo el agente, la recompensa en el _Pong_ oscila, aproximadamente, en el rango de valores **[-20, 20]**. La estrategia óptima de un agente estaría alrededor de una media de recompensa de 20.

- **NOTA IMPORTANTE**: Si el agente no llegara a aprender una estrategia ganadora, responder sobre la mejor estrategia obtenida.

In [35]:
!pip install keras-rl==0.4.2
!pip install tensorflow==1.13.1
!pip install gym
!pip install gym[atari]
!pip install h5py


#creo son necesarias, pero no se ha comprobado que en su ausencia falle
!pip install jupyter
!pip install torch

In [0]:
from PIL import Image

import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [37]:
ENV_NAME = 'PongDeterministic-v0'

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
nb_actions = env.action_space.n

# random seed
np.random.seed(123)
env.seed(123)

[123, 151010689]

In [0]:
# Define the input shape to resize the screen
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

# This processor will be similar to the Atari processor
class PongProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        
        img = Image.fromarray(observation)
        # resize and convert to grayscale
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [0]:
processor = PongProcessor()

In [0]:
def createModel():
    input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE

    model = Sequential()

    model.add(Permute((2, 3, 1), input_shape=input_shape))

    model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))

    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    return model

In [41]:
model = createModel()
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_3 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_13 (Activation)   (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_14 (Activation)   (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activation_15 (Activation)   (None, 7, 7, 64)         

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)

In [0]:
dqn = DQNAgent(
    nb_actions=nb_actions, 
    model=model, 
    policy=policy, 
    memory=memory,
    processor=processor, 
    nb_steps_warmup=50000, 
    target_model_update=10000,
    gamma=.99,
    train_interval=20)

In [0]:
#compilar modelo
optimizer = Adam(lr=1e-3)
dqn.compile(optimizer, metrics=['mae'])

In [0]:
#callback
checkpoint_weights_filename = 'dqn_' + ENV_NAME + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(ENV_NAME)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=50000)]
callbacks += [FileLogger(log_filename, interval=100)]

In [46]:
# Training part
#dqn.fit(env, callbacks=callbacks, nb_steps=500000, log_interval=10000, visualize=False)

Training for 500000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 45s 5ms/step - reward: -0.0219
10 episodes - episode_reward: -20.100 [-21.000, -17.000] - ale.lives: 0.000

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: -0.0200
11 episodes - episode_reward: -19.727 [-21.000, -17.000] - ale.lives: 0.000

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: -0.0219
10 episodes - episode_reward: -20.400 [-21.000, -19.000] - ale.lives: 0.000

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 47s 5ms/step - reward: -0.0237
12 episodes - episode_reward: -20.333 [-21.000, -18.000] - ale.lives: 0.000

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 45s 5ms/step - reward: -0.0238
11 episodes - episode_reward: -20.727 [-21.000, -20.000] - ale.lives: 0.000

Interval 6 (50000 steps 

In [0]:
weights_filename = 'dqn_' + ENV_NAME + '_weights.h5f'

dqn.save_weights(weights_filename, overwrite=True)

In [50]:
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: -21.000, steps: 764
Episode 2: reward: -21.000, steps: 764
Episode 3: reward: -21.000, steps: 764
Episode 4: reward: -21.000, steps: 764
Episode 5: reward: -21.000, steps: 764
Episode 6: reward: -21.000, steps: 764
Episode 7: reward: -21.000, steps: 764
Episode 8: reward: -21.000, steps: 764
Episode 9: reward: -21.000, steps: 764
Episode 10: reward: -21.000, steps: 764
